<a href="https://colab.research.google.com/github/feelpp/book.feelpp.org/blob/master/pyfeelpp_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feel++ Notebook

In this notebook, we 


* install Feel++
* test it some core functionalities
* retrieve results to visualize them locally in Paraview



## Installation of Feel++
we start with installing Feel++ in colab


*   set the proper sources
*   add the bintray key
*   update the repo 
*   install python3-feelpp 





In [3]:
!echo "deb https://dl.bintray.com/feelpp/ubuntu bionic latest" | tee -a /etc/apt/sources.list
!wget -qO - https://bintray.com/user/downloadSubjectPublicKey?username=bintray | apt-key add -
!apt update
!apt install feelpp-quickstart feelpp-data
!apt install python3-mpi4py python3-feelpp ssh 

deb https://dl.bintray.com/feelpp/ubuntu bionic latest
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://dl.bintray.com/feelpp/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://dl.bintray.com/feelpp/ubuntu bionic Release [3,560 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ 

## Core Test

In [4]:
import sys
import feelpp
e=feelpp.Environment(sys.argv)
e.setConfigFile("/usr/share/feelpp/data/testcases/quickstart/cases/triangle/triangle.cfg")
print(e.numberOfProcessors())
print("isMasterRank:",e.isMasterRank() )

1
isMasterRank: True


## Mesh Test

In [5]:
geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}


def run( m, geofile ):
    if e.isMasterRank():
        print("mesh dim:", m.dimension())
    
    m=feelpp.load(m,geofile,0.1)
    if e.isMasterRank():
        print("mesh ",m.dimension(),"D nelts:", m.numGlobalElements() )
        print("mesh ",m.dimension(),"D nfaces:", m.numGlobalFaces() )
        print("mesh ",m.dimension(),"D hmin:", m.hMin())
        print("mesh ",m.dimension(),"D havg:", m.hAverage())
        print("mesh ",m.dimension(),"D hmax:", m.hMax())
        print("mesh ",m.dimension(),"D measure:", m.measure())
    
    r = feelpp.elements(m)
    print("mesh elts:", feelpp.nelements(r,True))
    r = feelpp.boundaryfaces(m)
    print("mesh boundary faces:", feelpp.nfaces(r,True))

run( feelpp.mesh(dim=2), geo['2'] )
run( feelpp.mesh(dim=3,realdim=3), geo['3'] )


mesh dim: 2
mesh  2 D nelts: 5140
mesh  2 D nfaces: 8015
mesh  2 D hmin: 0.07512597426289651
mesh  2 D havg: 0.1025627812243611
mesh  2 D hmax: 0.12930415826864228
mesh  2 D measure: 20.799999999999933
mesh elts: 5140
mesh boundary faces: 610
mesh dim: 3
mesh  3 D nelts: 93724
mesh  3 D nfaces: 200042
mesh  3 D hmin: 0.09999999999930598
mesh  3 D havg: 0.16518630031776135
mesh  3 D hmax: 0.23047955624802838
mesh  3 D measure: 20.79999999999997
mesh elts: 93724
mesh boundary faces: 25188


## FunctionSpace Test

In [6]:
geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}

def run( m, geo ):
    m2d = feelpp.load(m,geo,0.1)
    Xh=feelpp.functionSpace(mesh=m2d)

    if e.isMasterRank():
        print("Xh basisname: ", Xh.basisName())
        print("Xh nDof: ", Xh.nDof())
        print("Xh nLocalDof: ", Xh.nLocalDof())
        print("Xh nLocalDofWithGhost: ", Xh.nLocalDofWithGhost())
        print("Xh nLocalDofWithoutGhost: ", Xh.nLocalDofWithoutGhost())

    m3=Xh.mesh()

    assert m3==m2d

    u=Xh.element()
    u.on(range=feelpp.elements(m2d),expr=feelpp.expr("x:x"))

    assert u.functionSpace() == Xh
    assert u.size() == Xh.nDof()

run( feelpp.mesh(dim=2), geo['2'] )
run( feelpp.mesh(dim=3,realdim=3), geo['3'] )


Xh basisname:  lagrange
Xh nDof:  2870
Xh nLocalDof:  2870
Xh nLocalDofWithGhost:  2870
Xh nLocalDofWithoutGhost:  2870
Xh basisname:  lagrange
Xh nDof:  21755
Xh nLocalDof:  21755
Xh nLocalDofWithGhost:  21755
Xh nLocalDofWithoutGhost:  21755


## Exporter Test

In [21]:

geo={
    '2':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0],
    '3':feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp3d/feelpp3d.geo}", worldComm=feelpp.Environment.worldCommPtr() )[0]
}

def run( m, geo ):
    mesh = feelpp.load(m,geo,0.1)

    Xh=feelpp.functionSpace( space="Pch", mesh=mesh, order=1 )
    P0h = feelpp.functionSpace( space="Pdh", mesh=mesh, order=0 )
    #u=Xh.elementFromExpr("{sin(2*pi*x)*cos(pi*y)}:x:y")
    u=Xh.element()
    u.on(range=feelpp.elements(mesh),expr=feelpp.expr("x*x:x"))

    e = feelpp.exporter(mesh=mesh,name="feelpp"+str(m.dimension())+"d")
    e.addScalar("un", 1.)
    e.addP1c("u",u)
    e.addP0d("pid",feelpp.pid( P0h ))
    e.save()

run( feelpp.mesh( dim=2 ), geo['2'] )
run( feelpp.mesh( dim=3, realdim=3 ), geo['3'] )


In [23]:
!ls -lrt exports/ensightgold/feelpp2d
!ls -lrt exports/ensightgold/feelpp2d

total 148
-rw-r--r-- 1 root root 96852 Nov  6 06:59 feelpp2d.geo.0001
-rw-r--r-- 1 root root 11724 Nov  6 06:59 feelpp2d.u.scl.0001
-rw-r--r-- 1 root root   105 Nov  6 06:59 feelpp2d.timeset
-rw-r--r-- 1 root root 20804 Nov  6 06:59 feelpp2d.pid.scl.0001
-rw-r--r-- 1 root root   229 Nov  6 06:59 feelpp2d-paraview-1.sos
-rw-r--r-- 1 root root   335 Nov  6 06:59 feelpp2d.case
-rw-r--r-- 1 root root   249 Nov  6 06:59 feelpp2d-1.sos
total 148
-rw-r--r-- 1 root root 96852 Nov  6 06:59 feelpp2d.geo.0001
-rw-r--r-- 1 root root 11724 Nov  6 06:59 feelpp2d.u.scl.0001
-rw-r--r-- 1 root root   105 Nov  6 06:59 feelpp2d.timeset
-rw-r--r-- 1 root root 20804 Nov  6 06:59 feelpp2d.pid.scl.0001
-rw-r--r-- 1 root root   229 Nov  6 06:59 feelpp2d-paraview-1.sos
-rw-r--r-- 1 root root   335 Nov  6 06:59 feelpp2d.case
-rw-r--r-- 1 root root   249 Nov  6 06:59 feelpp2d-1.sos


In [20]:
# Save to your local computer
from google.colab import files
!zip -r feelpp.zip exports/ensightgold/
files.download('feelpp.zip')

  adding: exports/ensightgold/ (stored 0%)
  adding: exports/ensightgold/feelpp3d/ (stored 0%)
  adding: exports/ensightgold/feelpp3d/feelpp3d.pid.scl.0001 (deflated 100%)
  adding: exports/ensightgold/feelpp3d/feelpp3d.case (deflated 37%)
  adding: exports/ensightgold/feelpp3d/feelpp3d.timeset (deflated 30%)
  adding: exports/ensightgold/feelpp3d/feelpp3d.geo.0001 (deflated 41%)
  adding: exports/ensightgold/feelpp3d/feelpp3d.u.scl.0001 (deflated 23%)
  adding: exports/ensightgold/feelpp3d/feelpp3d-paraview-1.sos (deflated 24%)
  adding: exports/ensightgold/feelpp3d/feelpp3d-1.sos (deflated 28%)
  adding: exports/ensightgold/feelpp2d/ (stored 0%)
  adding: exports/ensightgold/feelpp2d/feelpp2d.pid.scl.0001 (deflated 100%)
  adding: exports/ensightgold/feelpp2d/feelpp2d-paraview-1.sos (deflated 24%)
  adding: exports/ensightgold/feelpp2d/feelpp2d-1.sos (deflated 28%)
  adding: exports/ensightgold/feelpp2d/feelpp2d.case (deflated 37%)
  adding: exports/ensightgold/feelpp2d/feelpp2d.u.sc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>